# Homework 5: The Normal Distribution and the Central Limit Theorem

## Due Tuesday, February 25th at 11:59PM

Welcome to Homework 5! This homework will cover:

* [CIT 14.2](https://www.inferentialthinking.com/chapters/14/2/Variability.html): Variability
* [CIT 14.3](https://www.inferentialthinking.com/chapters/14/3/SD_and_the_Normal_Curve.html): The Standard Deviation (SD) and the Normal Curve 
* [CIT 14.4](https://www.inferentialthinking.com/chapters/14/4/Central_Limit_Theorem.html): The Central Limit Theorem
* [CIT 14.5](https://www.inferentialthinking.com/chapters/14/5/Variability_of_the_Sample_Mean.html): The Variability of the Sample Mean
* [CIT 14.6](https://inferentialthinking.com/chapters/14/6/Choosing_a_Sample_Size.html): Choosing a Sample Size

### Instructions

You are given six slip days throughout the quarter to extend deadlines. See the syllabus for more details. With the exception of using slip days, late work will not be accepted unless you have made special arrangements with your instructor.

**Important**: For homeworks, the `otter` tests don't usually tell you that your answer is correct. More often, they help catch careless mistakes. It's up to you to ensure that your answer is correct. If you're not sure, ask someone (not for the answer, but for some guidance about your approach). These are great questions for office hours (see the schedule on the [Calendar](https://dsc10.com/calendar)) or Ed. Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged.

In [ ]:
# Please don't change this cell, but do make sure to run it
import babypandas as bpd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import otter
grader = otter.Notebook()

from IPython.display import IFrame
def show_clt_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vTcJd3U1H1KoXqBFcWGKFUPjZbeW4oiNZZLCFY8jqvSDsl4L1rRTg7980nPs1TGCAecYKUZxH5MZIBh/embed?start=false&loop=false&delayms=3000&rm=minimal"
    width = 700
    height = 370
    display(IFrame(src, width, height))

## 1. Testing the Central Limit Theorem: Coin Flips and Ramen Ratings 🍜


The Central Limit Theorem tells us that the probability distribution of the sum or mean of a large random sample drawn with replacement is roughly normal, *regardless of the distribution of the population from which the sample is drawn*.

That's a pretty big claim, but the theorem doesn't stop there. It further states that, if we're using the mean as our statistic, the standard deviation of this normal distribution is given by $$\text{SD of Distribution of Possible Sample Means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$$

 In other words, suppose we start with *any distribution* that has standard deviation $\sigma$, take a sample of size $n$ (where $n$ is a large number) from that distribution with replacement, and compute the mean of that sample. If we repeat this procedure many times, then those sample means will have a normal distribution with standard deviation $\frac{\sigma}{\sqrt{n}}$.

That's an even bigger claim than the first one! The proof of the theorem is beyond the scope of this class, but we've seen examples in lecture of this formula in action, such as when we looked at flight delay data.

Run the cell below to see a short presentation that describes the CLT at a high level.

In [ ]:
show_clt_slides()

In this problem, we will be exploring some new data to see the CLT in action.

The CLT only applies when sample sizes are "sufficiently large." This isn't a very precise statement. Is 10 large?  How about 50?  The truth is that it depends both on the original population distribution and just how "normal" you want the result to look. Let's use a simulation to get a feel for how the distribution of the sample mean changes as the sample size increases.

Consider a coin flip. If we say tails is 0 and heads is 1, then there's a 50% chance of getting a 0 and a 50% chance of getting a 1. A histogram of this distribution is shown below.

In [ ]:
(
    bpd.DataFrame().assign(outcomes=[0, 1])
    .plot(kind='hist', density=True, bins=[-0.5, 0.5, 1.5], ec='w')
)
plt.xticks([0, 1]);

This distribution is not roughly normal!

The act of flipping a coin many times and computing the proportion of heads is equivalent to drawing a large sample from the above distribution with replacement and computing its mean. Since the proportion of heads in a sample of coin tosses is equal to the mean of that sample, the Central Limit Theorem should apply if we repeatedly toss many coins and compute the proportion of tosses that were heads. Let's try it out and see for ourselves!

**Question 1.1.** Complete the implementation of the function `simulate_sample_n`. It should take in a sample size, `n`, and should repeat, 5000 times, the process of:
- simulating `n` flips of a fair coin, and
- finding the proportion of flips that were heads.

`simulate_sample_n` should return an array that contains 5000 sample proportions, using the process outlined above.

In [ ]:
def simulate_sample_n(n):
    ...
simulate_sample_n(5)

In [ ]:
grader.check("q1_1")

The code below will use the function you just defined to plot the empirical distribution of the sample mean for several different sample sizes. We saw something similar in [Lecture 17](https://dsc10.com/resources/lectures/lec17/lec17.html#Changing-the-sample-size).

In [ ]:
bins = np.arange(-0.01, 1.05, 0.02)

for sample_size in np.array([2, 5, 10, 20, 50, 100, 200, 400]):
    bpd.DataFrame().assign(**{'Sample_Size:{}'.format(sample_size) : simulate_sample_n(int(sample_size))}) \
                   .plot(kind='hist', density=True, ec='w', bins=bins, 
                         title=f'Sample Size {sample_size}', legend=None, figsize=(5, 3));
    plt.xlim(-0.01, 1.05)
    plt.ylim(0, 25);

You can see that even for samples of size 10, the distribution of sample proportions looks roughly bell-shaped. When we increase the sample size to 50, the resulting distribution looks quite bell-shaped. Note also that as the sample sizes increases, the distributions of sample proportions become narrower.

Now we will test the second claim of the CLT: that the SD of the distribution of the sample mean is the SD of the original distribution, divided by the square root of the sample size.

$$\text{SD of Distribution of Possible Sample Means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$$

To do this, we'll work with a dataset of ramen product ratings, as reviewed by [The Ramen Rater](https://www.theramenrater.com/). Below, we save this data in the `ramen` DataFrame. We'll treat `ramen` as our population, and we'll take samples directly from it. Note that the `'Stars'` column contains the rating given to each product, which ranges from 0 to 5 stars. We'll be focusing on the mean `'Stars'` rating for various samples from `ramen`.

We've computed the standard deviation of the ramen ratings for you; you will need to use it in the next question.

In [ ]:
ramen = bpd.read_csv('data/ramen.csv')
ramen

In [ ]:
ramen_std = np.std(ramen.get('Stars'))
ramen_std

**Question 1.2.** Complete the implementation of the function `predict_sd`. It should take in a sample size `n` and return the predicted standard deviation (according to the CLT) of the sample mean's distribution, for samples of size `n` taken from the `ramen` DataFrame.

***Hint:*** **Do not** use or modify your code from `simulate_sample_n`. You should not actually take any samples to answer this question.

In [ ]:
def predict_sd(n):
    ...

predict_sd(10)

In [ ]:
grader.check("q1_2")

**Question 1.3.** Complete the implementation of the function `empirical_sd`. It should take in a sample size `n`, draw 1,000 samples of size `n` from `ramen` with replacement, calculate the mean of each sample, and return the **standard deviation of the distribution of the sample means**.

***Hint:*** This function will be similar to the `simulate_sample_n` function you wrote earlier.

In [ ]:
def empirical_sd(n): 
    sample_means = np.array([])
    ...
    return np.std(sample_means)

In [ ]:
grader.check("q1_3")

The cell below will plot the predicted SDs (computed by your `predict_sd` function) and empirical SDs (computed by your `empirical_sd` function) for various sample sizes. It may take a few moments to run.

In [ ]:
sd_df = bpd.DataFrame().assign(Sample_Size = np.arange(10, 101, 10))
predicted = sd_df.get('Sample_Size').apply(predict_sd)
empirical = sd_df.get('Sample_Size').apply(empirical_sd)
sd_df = sd_df.assign(Predicted_SD = predicted, Empirical_SD = empirical)
ax = sd_df.plot(kind='scatter',x='Sample_Size', y='Empirical_SD',label='Empirical_SD', color='red', alpha=0.6, s=100, figsize=(10, 5));
ax = sd_df.plot(kind='scatter',x='Sample_Size', y='Predicted_SD',label='Predicted_SD', color='blue', alpha=0.6, s=100, ax=ax)
ax.set_ylabel('Standard Deviation');

It appears that the formula $$\text{SD of Distribution of Possible Sample Means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$$ matches what we see in practice!

## 2. Til' Death Duo Part 🦉


<center><img src=images/duolingo.jpg width=250><br></center>

The green owl that once gave us the ultimatum to "Spanish or Vanish" has now vanished himself. Earlier this month, the language learning app Duolingo [announced on Instagram](https://www.instagram.com/p/DF79IjgxfuE/?igsh=NTc4MTIwNjQ2YQ%3D%3D) that their beloved mascot, Duo the Owl, has died. The cause of death has not yet been revealed, and companies and celebrities alike are sharing their thoughts on the situation. [Netflix](https://x.com/netflix/status/1889390241498497189) explained that Duo was eliminated in the Korean series, "Squid Game". [Dua Lipa](https://x.com/DUALIPA/status/1889661358955733300) expressed her condolences in a post on X which said "Til' death duo part" (the title of this question). But from the information gathered, the most likely cause of death is that Duo was hit by a [Tesla Cybertruck](https://www.instagram.com/reel/DF-peENxg3I/?igsh=NTc4MTIwNjQ2YQ==).

To pay our respects to Duo, we can learn more about his appearances all across the world. Every year, Duolingo publishes data about the most popular language course in each country. We've have downloaded the data and loaded it into a DataFrame called `duolingo`.

In [ ]:
duolingo = bpd.read_csv('data/duolingo.csv')
duolingo

Each row corresponds to a country, and the most popular language being learned in that country on Duolingo, which is stored in `'most_pop_language'`.

In the cell below, we compute the distribution of the `'most_pop_language'` column in `duolingo`.

In [ ]:
duolingo_languages = duolingo.groupby('most_pop_language').count()
duolingo_languages = (duolingo_languages.assign(count=duolingo_languages.get('country'))
                                        .sort_values(by='count', ascending=False)
                                        .get(['count']))
duolingo_languages

Based on the data, the most common `'most_pop_language'` is `'English'`, with 134 out of 193 countries having it as their most learned language on Duolingo, which is $\frac{134}{193} \approx .69$. `'Spanish'` and `'French'` round out the podium as the second and third most common languages, respectively. 

We'll treat `duolingo` as our population, and we'll repeatedly draw samples from this population to compute two statistics — the sample proportion of each `'most_pop_language'` and the sample count of each `'most_pop_language'`. Our goal is to better understand the relationship between sample proportions, sample counts, and full distributions.

**Question 2.1.** 10,000 times, draw a [**simple random sample**](https://dsc10.com/resources/lectures/lec13/lec13.html#Simple-random-sample) of 100 countries from the `duolingo` DataFrame. For each sample, calculate the proportion of countries with a `'most_pop_language'` of `'English'`. Store all 10,000 calculated proportions in the array `props`.

In [ ]:
props = ...

In [ ]:
grader.check("q2_1")

Once you've defined `props`, run the following cell to see the distribution of values in `props`.

In [ ]:
bpd.DataFrame().assign(props=props).plot(kind='hist', density=True, ec='w', bins=11, figsize=(10, 5));

**Question 2.2.** What _should_ the mean of `props` be, according to the Central Limit Theorem? Store this value in `theoretical_props_mean`. You should not access the data in `props` when calculating `theoretical_props_mean`, because `props` contains the empirical results of an experiment; instead, only refer to `duolingo`. 

Then, calculate the actual mean of `props` and store this value in `actual_props_mean`. Here, you should refer to `props`.

In [ ]:
theoretical_props_mean = ...
actual_props_mean = ...

# Don't change the following line.
print(f'The Central Limit Theorem states that the mean of `props` should be {theoretical_props_mean}.\nThe actual mean of `props` is {actual_props_mean}.')

In [ ]:
grader.check("q2_2")

**Question 2.3.** Again, 10,000 times, draw a simple random sample of 100 countries from the population of languages in `duolingo`. This time, for each sample, compute the **number** of countries with a `'most_pop_language'` of `'English'`. Store all 10,000 counts in the array `counts`.

In [ ]:
counts = ...
counts

In [ ]:
grader.check("q2_3")

Once you've defined `counts`, run the following cell to see the distribution of values in `counts`.

In [ ]:
bpd.DataFrame().assign(counts=counts).plot(kind='hist', density=True, ec='w', bins=11, figsize=(10, 5));

**Question 2.4.** What _should_ the mean of `counts` be, according to the Central Limit Theorem? Store this value in `theoretical_counts_mean`. Again, you should not access the data in `counts` when calculating `theoretical_counts_mean`; instead, only refer to `duolingo`.

Then, calculate the actual mean of `counts` and store this value in `actual_counts_mean`. Here, you should refer to `counts`.

***Hint:*** A count is not a mean, so the Central Limit Theorem as we've seen it does not *directly* apply. However, you can use `theoretical_props_mean` from Question 2.2 to help you find `theoretical_counts_mean`. 

In [ ]:
theoretical_counts_mean = ...
actual_counts_mean = ...

# Don't change the following line.
print(f'The Central Limit Theorem states that the mean of `counts` should be {theoretical_counts_mean}.\nThe actual mean of `counts` is {actual_counts_mean}.')

In [ ]:
grader.check("q2_4")

**Question 2.5.** Let's explore how the histogram of counts changes when we take samples of different sizes. 

Complete the implementation of the function `count_most_pop_language`. It should take in an integer `size` and do the following:
- 10,000 times, draw a simple random sample of the given `size` from `duolingo`.
- For each sample, calculate the number of countries with a `'most_pop_language'` of `'English'`.
- Return an array with these 10,000 counts.

This requires generalizing your code from Question 2.3.

In [ ]:
def count_most_pop_language(size):
    ...

In [ ]:
grader.check("q2_5")

The code below will use the function you just defined to plot the empirical distribution of the count of countries with a `'most_pop_language'` of `'English'`, for several different sample sizes. It may take a few moments to run.

In [ ]:
for sample_size in [10, 25, 50, 100]:
    bpd.DataFrame().assign(types=count_most_pop_language(sample_size)) \
                   .plot(kind='hist', density=True, ec='w', bins = np.arange(0, 100, 2), 
                         title=f'Sample Size {sample_size}', legend=None, figsize=(5, 3))

**Question 2.6.** Using the plots above, what do you notice about the relationship between sample size and the mean of the empirical distribution of the count of countries with a `'most_pop_language'` of `'English'`? Similarly, what do you notice about the relationship between sample size and the standard deviation of the empirical distribution of the count of countries with a `'most_pop_language'` of `'English'`? 

Assign `q2_6` to a **list** of numbers 1 through 4 corresponding to the true statements below.
1. As the sample size increases, the mean of the distribution of counts increases.
1. As the sample size increases, the mean of the distribution of counts decreases.
1. As the sample size increases, the standard deviation of the distribution of counts increases.
1. As the sample size increases, the standard deviation of the distribution of counts decreases.

In [ ]:
q2_6 = ...

In [ ]:
grader.check("q2_6")

## 3. Drake vs. Kendrick Lamar 🎤👑

Last summer, the [Drake-Kendrick Lamar rap feud](https://en.wikipedia.org/wiki/Drake%E2%80%93Kendrick_Lamar_feud) captured public attention as the two artists traded insults through targeted diss tracks. This drama was recently rehashed with [Kendrick's Super Bowl performance](https://youtu.be/KDorKy-13ak) a few weeks ago. 

The general consensus online is that Kendrick "won" the battle, but who are UCSD students crowning the winner? To answer this question, Sofia decided to survey some of the [43,533 current UCSD students](https://ir.ucsd.edu/third-week/index.html), asking them if they believe Drake or Kendrick Lamar is the better rapper. Out of the 580 randomly sampled students, 203 said they prefer Drake and 377 said they prefer Kendrick Lamar. We've recorded this information in the DataFrame below.

In [ ]:
# Run this cell, but don't change it.
survey = bpd.DataFrame().assign(
    Opinion=np.array(['Drake', 'Kendrick Lamar']),
    Count=np.array([203, 377]))
sample_size = survey.get('Count').sum()
survey_results = survey.assign(
    Proportion=survey.get('Count') / sample_size)
survey_results

Next, Sofia used 1,000 bootstrap resamples to compute a confidence interval for the proportion of all UCSD students who prefer `'Kendrick Lamar'`.  Run the next cell to see the empirical distribution of this proportion in the 1,000 resamples.

Note that we're using `np.random.multinomial` to do the resampling here, since each element of the resample is either 1 (`'Kendrick Lamar'`) or 0 (`'Drake'`) with known probabilities. This accomplishes the same thing as using `.sample` with `replace=True`, but is much faster.

In [ ]:
kendrick_proportions = np.array([])
for i in np.arange(1000):
    resample = np.random.multinomial(sample_size, survey_results.get('Proportion')) / sample_size
    kendrick_proportions = np.append(kendrick_proportions, resample[1])
bpd.DataFrame().assign(kendrick_proportions = kendrick_proportions) \
                .plot(kind='hist', density=True, ec='w', bins=np.arange(0.55, 0.75, 0.01), figsize=(10,5));

Recall, the Central Limit Theorem says

$$\text{SD of Distribution of Possible Sample Means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$$

Furthermore, in any collection of numbers where the only unique values are 0 and 1, there is a simple formula for the standard deviation of the collection:

$$\text{SD of Collection of 0s and 1s} = \sqrt{(\text{Proportion of 0s in Collection}) \times (\text{Proportion of 1s in Collection})}$$

Note that samples and populations are both possible examples of "collections." 

(You're not responsible for deriving this formula, but if you're curious, it's possible to do so just by using the definition of standard deviation and a little algebra!)

Since you'll need the numbers in it to answer Question 3.1, here's `survey_results` once again:

In [ ]:
survey_results

**Question 3.1.**
**Without accessing the data in `kendrick_proportions` in any way**, compute an approximation of the standard deviation of the array `kendrick_proportions` and assign it to the variable `approximate_sd`.

Instead of using `kendrick_proportions` directly, use **both** the Central Limit Theorem and the standard deviation formula above. Since you don't know the true proportions of 0s and 1s in the population, use the proportions in the sample instead (since they're likely to be similar), which are provided in the DataFrame `survey_results`.

In [ ]:
approximate_sd = ...
approximate_sd

In [ ]:
grader.check("q3_1")

**Question 3.2.** Compute the actual standard deviation of the array `kendrick_proportions`. Your answer should be close to your answer from Question 3.1.

In [ ]:
exact_sd = ...
exact_sd

In [ ]:
grader.check("q3_2")

Since you'll need the numbers in it to answer Question 3.3, here's `survey_results` once again:

In [ ]:
survey_results

**Question 3.3.**
**Without accessing `kendrick_proportions` in any way**, compute an approximate 95% confidence interval for the proportion of students who prefer `'Kendrick Lamar'`.

The cell below `grader.check("q3_3")` draws your interval in gold below the histogram of `kendrick_proportions`; use that to verify that your answer looks right.

***Hint:*** In the past, we've used `np.percentile` on the array of bootstrapped estimates to find the bounds for the confidence interval. Now, **we're not allowed to use the bootstrapped distribution**, so we can't do it that way. But we don't need to! 

- The Central Limit Theorem tells us that the distribution of the sample mean is normal with a certain mean and standard deviation.
- `survey_results` provides an estimate of this mean; you can use that number here.
- In Question 3.1, you estimated this standard deviation (without using `kendrick_proportions`) and saved it to the variable `approximate_sd`; you can use that here as well.
- We also know that 95% of the area of the normal distribution falls within a certain number of standard deviations from the mean.

In [ ]:
lower_limit = ...
upper_limit = ...

# Your interval is:
[lower_limit, upper_limit]

In [ ]:
grader.check("q3_3")

In [ ]:
# Run this cell to plot your confidence interval.
bpd.DataFrame().assign(kendrick_proportions = kendrick_proportions).plot(kind='hist', density=True, ec='w', bins=np.arange(0.55, 0.75, 0.01), figsize=(10, 5));
plt.plot([upper_limit, lower_limit], [0, 0], color='gold', linewidth=10, label='Normal CI');
plt.legend();

Your confidence interval should make it clear that we're pretty confident that most students prefer `'Kendrick Lamar'`. This makes sense, as the proportion of people choosing `'Kendrick Lamar'` in the sample was high. 

Sofia is considering redoing the survey with a larger sample to estimate with greater precision the population proportion of students who prefer `'Kendrick Lamar'`. She wants the **standard deviation of the distribution of the sample mean to be 0.004** (or less).  She will need to take a new sample that's large enough to achieve that. Polling is time-consuming, so the sample also shouldn't be bigger than necessary.

Instead of making the conservative assumption that the population standard deviation is 0.5 (the largest possible SD of a collection of 0s and 1s), she decides to assume that it's equal to the standard deviation of her first sample. That is,

$$\text{Population SD} \approx \text{Sample SD} = \sqrt{(\text{Proportion of 0s in Sample}) \times (\text{Proportion of 1s in Sample})}$$

Under that assumption, she computes the smallest sample size necessary in order to be confident that the standard deviation of the distribution of the sample mean is at most 0.004.

**Question 3.4.**
What sample size did Sofia find? Assign your answer to the variable `new_sample_size`, which should be of type `int`.

Use the fact that $$\text{SD of Distribution of Possible Sample Means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$$

***Hints:***
- There is only one unknown in the equation above.
- Think about how you should round your answer to satisfy the constraints of the problem.

In [ ]:
new_sample_size = ...
new_sample_size

In [ ]:
grader.check("q3_4")

**Question 3.5.** Suppose Sofia wants to be even more precise and take a sample of sufficient size such that the standard deviation of the sample mean distribution is 0.002. Is it possible for them to do this? Choose the best answer and explanation, then assign `q3_5` to either 1, 2, 3, or 4.

1. Yes. They can repeat the sample again until they comes across a sample with a standard deviation of 0.002.
1. Yes. Since the 0.002 is half of 0.004, the required sample size is half of `new_sample_size`.
1. Yes. Since the 0.002 is half of 0.004, the required sample size is twice as large as `new_sample_size`.
1. No, the sample size required to reach that sample mean standard deviation is larger than the number of students at UCSD (43,533).


In [ ]:
q3_5 = ...

In [ ]:
grader.check("q3_5")

## 4. Key Concepts 🔑

**Question 4.1.** How do we convert the value 36 to standard units if it comes from a dataset where the mean is 20 and the standard deviation is 3? Assign `q4_1` to either 1, 2, 3, or 4.



1.
$\dfrac{20-36}{3}$

2.
$\dfrac{{36-20}}{\sqrt{3}}$


3.
$\dfrac{36-20}{3}$

4.
$\dfrac{({20-36})^2}{3}$

In [ ]:
q4_1 = ...

In [ ]:
grader.check("q4_1")

**Question 4.2.** According to Chebyshev's inequality, for any dataset, at least half of the data falls within how many standard deviations of the mean? Assign the **smallest** correct answer to `q4_2`.

1. 1.35
2. 1.40
3. 1.45
4. 1.50

In [ ]:
q4_2 = ...

In [ ]:
grader.check("q4_2")

**Question 4.3.** Assign `q4_3` to a **list** of all statements below that are **always** true.



1. An empirical histogram of the sample mean of a large random sample drawn with replacement from a population will be roughly normal.
1. An empirical histogram of the sample median of a large random sample drawn with replacement from a population will be roughly normal.
1. If we know the mean and SD of a distribution, we can calculate a 68% confidence interval by stepping out one standard deviation from the mean in both directions.
1. For any distribution, 95% of the data falls within two standard deviations of the mean.
1. For any distribution, at least 95% of the data falls within five standard deviations of the mean.


In [ ]:
q4_3 = ...

In [ ]:
grader.check("q4_3")

**Question 4.4.** Consider drawing a large random sample with replacement from some population. Let $x$ be the sample size such that the standard deviation of the distribution of sample means is ${y}$. Choose the sample size required to guarantee that the standard deviation of the distribution of sample means is no more than $\dfrac{y}{4}$. Assign `q4_4` to either 1, 2, 3, or 4.

1. $2x$
1. $4x$
1. $8x$
1. $16x$

In [ ]:
q4_4 = ...

In [ ]:
grader.check("q4_4")

## Finish Line: Almost there, but make sure to follow the steps below to submit! 🏁

**_Citations:_** Did you use any generative artificial intelligence tools to assist you on this assignment? If so, please state, for each tool you used, the name of the tool (ex. ChatGPT) and the problem(s) in this assignment where you used the tool for help.

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

Please cite tools here.

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

Congratulations! You are done with Homework 5, the second-to-last homework of the quarter!

To submit your assignment:

1. Select `Kernel -> Restart & Run All` to ensure that you have executed all cells, including the test cells.
1. Read through the notebook to make sure everything is fine and all tests passed.
1. Run the cell below to run all tests, and make sure that they all pass.
1. Download your notebook using `File -> Download as -> Notebook (.ipynb)`, then upload your notebook to Gradescope.
1. Stick around while the Gradescope autograder grades your work. Make sure you see that all tests have passed on Gradescope.
1. Check that you have a confirmation email from Gradescope and save it as proof of your submission.

In [ ]:
grader.check_all()